In [0]:
# ============================================================================
# Camada Gold - Report Alertas Executivos - Magic: The Gathering
# Pipeline 100% PySpark DataFrame API
# ============================================================================

# ============================================================================
# BIBLIOTECAS UTILIZADAS
# ============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

# ============================================================================
# IMPORTAÇÃO DO MÓDULO UTILITÁRIO
# ============================================================================
%run ./gold_utils

# ============================================================================
# CONFIGURAÇÃO COM MÓDULO UTILITÁRIO
# ============================================================================
TABLE_NAME = "TB_REPORT_ALERTAS_EXECUTIVOS"

# Inicializar processador com configuração manual 
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")


config = create_manual_config(
    catalog_name="magic_the_gathering",  
    s3_bucket= get_secret("s3_bucket"),  
    s3_gold_prefix="magic_the_gathering/gold"
)
processor = GoldTableProcessor(TABLE_NAME, config)
spark = processor.spark


In [0]:
# ============================================================================
# CARREGAMENTO DE DADOS SILVER COM MÓDULO
# ============================================================================
dfs = processor.load_silver_data(['cards', 'prices'])
df_cards = dfs['cards']
df_prices = dfs['prices']

# ============================================================================
# JOIN PRINCIPAL E LIMPEZA DE DADOS
# ============================================================================
df_base = df_cards.join(df_prices, ["NME_CARD", "COD_SET"], "inner")


In [0]:
# ============================================================================
# ANÁLISE DE VARIAÇÕES PARA ALERTAS
# ============================================================================
# Agregação diária por carta
df_val = (
    df_base.groupBy(col("prices.DT_INGESTION").alias("DT_INGESTION"), "NME_CARD")
      .agg(sum("VLR_USD").alias("VALOR_ATUAL"))
)

# Calcular variação com valor anterior
window_card = Window.partitionBy("NME_CARD").orderBy("DT_INGESTION")
df_val = df_val.withColumn("VALOR_ANTERIOR", lag("VALOR_ATUAL").over(window_card)) \
               .withColumn("VARIACAO_PERC_VALOR", 
                          when(col("VALOR_ANTERIOR").isNotNull() & (col("VALOR_ANTERIOR") != 0),
                               (col("VALOR_ATUAL") - col("VALOR_ANTERIOR")) / col("VALOR_ANTERIOR"))
                          .otherwise(lit(None)))

# ============================================================================
# CLASSIFICAÇÃO DO TIPO DE VARIAÇÃO
# ============================================================================
df_val = df_val.withColumn(
    "TIPO_VARIACAO",
    when(col("VARIACAO_PERC_VALOR") > 0, "Valorização")
    .when(col("VARIACAO_PERC_VALOR") < 0, "Desvalorização")
    .otherwise("Estável")
)

# ============================================================================
# GERAÇÃO DE ALERTAS CRÍTICOS
# ============================================================================

# Alerta Executivo: Quedas Abruptas (>20%)
df_alertas = df_val.filter(col("VARIACAO_PERC_VALOR") < -0.2) \
    .withColumn("TIPO_ALERTA", lit("Queda Abrupta")) \
    .withColumn("SEVERIDADE", lit("Crítico")) \
    .withColumn("DESCRICAO", lit("Queda superior a 20% no valor da carta")) \
    .withColumn("DATA_REF", to_date(col("DT_INGESTION")))

# Alertas Adicionais: Valorizações Extremas (>50%)
df_alertas_alta = df_val.filter(col("VARIACAO_PERC_VALOR") > 0.5) \
    .withColumn("TIPO_ALERTA", lit("Valorização Extrema")) \
    .withColumn("SEVERIDADE", lit("Atenção")) \
    .withColumn("DESCRICAO", lit("Valorização superior a 50% no valor da carta")) \
    .withColumn("DATA_REF", to_date(col("DT_INGESTION")))



In [0]:
# ============================================================================
# CONSOLIDAÇÃO DOS ALERTAS
# ============================================================================
df_alertas_consolidados = df_alertas.union(df_alertas_alta)

# Adicionar métricas de impacto
df_alertas_consolidados = df_alertas_consolidados.withColumn("VALOR_ABSOLUTO_VARIACAO",
                                                           abs(col("VARIACAO_PERC_VALOR"))) \
                                                 .withColumn("IMPACTO_FINANCEIRO",
                                                           abs(col("VALOR_ATUAL") - col("VALOR_ANTERIOR")))

# Ranking de prioridade por data
window_prioridade = Window.partitionBy("DATA_REF").orderBy(abs(col("VARIACAO_PERC_VALOR")).desc())
df_alertas_consolidados = df_alertas_consolidados.withColumn("RANK_PRIORIDADE", 
                                                           row_number().over(window_prioridade))

# ============================================================================
# ESCRITA DA TABELA COM MÓDULO
# ============================================================================
processor.save_gold_table(df_alertas_consolidados, partition_cols=["DATA_REF"])

# ============================================================================
# FINALIZAÇÃO
# ============================================================================
print("TB_REPORT_ALERTAS_EXECUTIVOS modularizada criada com sucesso!")